In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2021-12-21 15:35:46.129367: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-21 15:35:46.129418: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.5.0
2.5.2


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
EPOCHS = 25

# Wikilabels
df = pd.read_csv('../data/wiki/wiki_name_race.csv')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df

# Additional features
sdf['name_first'] = sdf.name_first.str.title()

sdf.groupby('race').agg({'name_first': 'count'})

,name_first
race,
"Asian,GreaterEastAsian,EastAsian",5497
"Asian,GreaterEastAsian,Japanese",7333
"Asian,IndianSubContinent",7861
"GreaterAfrican,Africans",3672
"GreaterAfrican,Muslim",6242
"GreaterEuropean,British",41445
"GreaterEuropean,EastEuropean",8329
"GreaterEuropean,Jewish",10239
"GreaterEuropean,WestEuropean,French",12293


## Preprocessing the input data

In [3]:
# only last name will be use to train the model
sdf['name_last_name_first'] = sdf['name_last'] 

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = ['UNK']
words_list.extend([w[1] for w in words])
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1436
Max feature len = 70, Avg. feature len = 5


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [4]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

107097 train sequences
26775 test sequences
Pad sequences (samples x time)
X_train shape: (107097, 20)
X_test shape: (26775, 20)
13 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (107097, 13)
y_test shape: (26775, 13)


In [5]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 32)            45952     
_________________________________________________________________

2021-12-21 15:35:58.783127: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-21 15:35:58.783169: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-21 15:35:58.783187: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-server-2): /proc/driver/nvidia/version does not exist
2021-12-21 15:35:58.783373: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 13)                1677      
Total params: 130,061
Trainable params: 130,061
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/25


2021-12-21 15:35:59.169284: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-21 15:35:59.169847: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2250000000 Hz


3013/3013 [==============================] - 115s 32ms/step - loss: 1.8290 - accuracy: 0.4477 - val_loss: 1.3863 - val_accuracy: 0.5914
Epoch 2/25
3013/3013 [==============================] - 104s 35ms/step - loss: 1.3748 - accuracy: 0.5946 - val_loss: 1.3145 - val_accuracy: 0.6168
Epoch 3/25
3013/3013 [==============================] - 105s 35ms/step - loss: 1.3074 - accuracy: 0.6142 - val_loss: 1.2721 - val_accuracy: 0.6337
Epoch 4/25
3013/3013 [==============================] - 102s 34ms/step - loss: 1.2602 - accuracy: 0.6278 - val_loss: 1.2434 - val_accuracy: 0.6402
Epoch 5/25
3013/3013 [==============================] - 104s 34ms/step - loss: 1.2234 - accuracy: 0.6405 - val_loss: 1.2274 - val_accuracy: 0.6486
Epoch 6/25
3013/3013 [==============================] - 104s 35ms/step - loss: 1.1895 - accuracy: 0.6514 - val_loss: 1.2111 - val_accuracy: 0.6534
Epoch 7/25
3013/3013 [==============================] - 102s 34ms/step - loss: 1.1771 - accuracy: 0.6538 - val_loss: 1.2048 - val

## Confusion Matrix

In [7]:
p = model.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

837/837 - 8s
                                       precision    recall  f1-score   support

     Asian,GreaterEastAsian,EastAsian       0.82      0.76      0.79      1099
      Asian,GreaterEastAsian,Japanese       0.83      0.87      0.85      1467
             Asian,IndianSubContinent       0.70      0.67      0.69      1572
              GreaterAfrican,Africans       0.52      0.37      0.43       734
                GreaterAfrican,Muslim       0.57      0.55      0.56      1248
              GreaterEuropean,British       0.72      0.87      0.78      8289
         GreaterEuropean,EastEuropean       0.73      0.67      0.70      1666
               GreaterEuropean,Jewish       0.45      0.34      0.39      2048
  GreaterEuropean,WestEuropean,French       0.59      0.49      0.54      2459
GreaterEuropean,WestEuropean,Germanic       0.39      0.28      0.33       774
GreaterEuropean,WestEuropean,Hispanic       0.65      0.53      0.59      2082
 GreaterEuropean,WestEuropean,Italian 

## Save model

In [8]:
model.save('./wiki/lstm/wiki_ln_lstm.h5')

In [9]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./wiki/lstm/wiki_ln_vocab.csv', index=False, encoding='utf-8')

In [10]:
sdf.groupby('race').agg({'name_first': 'count'}).to_csv('./wiki/lstm/wiki_race.csv', columns=[])